In [ ]:
# @title 1. Instalación de Librerías
# @markdown Ejecuta esta celda para instalar las librerías necesarias para el análisis.

!pip install geoip2 ipwhois prettytable




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 12.2 MB/s eta 0:00:00


In [ ]:
# @title 2. Carga del Fichero de Log
# @markdown Ejecuta esta celda para subir el fichero `vpn_access_v3.log` desde tu ordenador.

from google.colab import files

print("Por favor, sube el fichero 'vpn_access_v3.log':")
uploaded_log = files.upload()

# Guardamos el nombre del fichero para usarlo después
if uploaded_log:
    log_filename = list(uploaded_log.keys())[0]
    print(f"\n¡Fichero '{log_filename}' subido con éxito!")




Por favor, sube el fichero 'vpn_access_v3.log':


Saving vpn_access.log to vpn_access (3).log

¡Fichero 'vpn_access (3).log' subido con éxito!


In [ ]:
# @title 3. Carga de la Base de Datos MaxMind (Country)
# @markdown Ejecuta esta celda para subir tu fichero de base de datos `GeoLite2-Country.mmdb`.

from google.colab import files

print("\nPor favor, sube el fichero 'GeoLite2-Country.mmdb':")
uploaded_db = files.upload()

# Guardamos el nombre del fichero para usarlo después
if uploaded_db:
    db_filename = list(uploaded_db.keys())[0]
    print(f"\n¡Fichero '{db_filename}' subido con éxito!")





Por favor, sube el fichero 'GeoLite2-Country.mmdb':


Saving GeoLite2-Country.mmdb to GeoLite2-Country (1).mmdb

¡Fichero 'GeoLite2-Country (1).mmdb' subido con éxito!


In [ ]:
# @title 4. Carga de la Base de Datos MaxMind (City) - Para Viajes Imposibles
# @markdown Para el análisis de 'viajes imposibles', necesitamos las coordenadas (latitud/longitud), que se encuentran en la base de datos de **Ciudad**.
# @markdown
# @markdown Ejecuta esta celda para subir tu fichero de base de datos `GeoLite2-City.mmdb`.

from google.colab import files

print("\nPor favor, sube el fichero 'GeoLite2-City.mmdb':")
uploaded_db_city = files.upload()

# Guardamos el nombre del fichero para usarlo después
if uploaded_db_city:
    db_filename_city = list(uploaded_db_city.keys())[0]
    print(f"\n¡Fichero '{db_filename_city}' subido con éxito!")


Por favor, sube el fichero 'GeoLite2-City.mmdb':


In [ ]:
# @title 4. Extracción de Datos del Log
# @markdown Esta celda lee el fichero de log y extrae la información clave: para cada IP, qué usuarios se conectaron y con qué estado (SUCCESS, FAILED, etc.).
# @markdown Esta información se guardará para ser utilizada por las celdas de análisis posteriores.

import csv
from collections import defaultdict

# Este diccionario almacenará la información principal:
# Estructura: {'ip_address': {'user1': {'SUCCESS', 'FAILED'}, 'user2': {'SUCCESS'}}}
ip_user_status_map = defaultdict(lambda: defaultdict(set))

try:
    with open(log_filename, mode='r', encoding='utf-8') as f:
        next(f) # Saltamos la cabecera
        reader = csv.reader(f)
        for row in reader:
            if len(row) >= 4:
                timestamp, username, ip_address, status = row
                ip_user_status_map[ip_address][username].add(status)
    print(f"Análisis del log completado. Se encontraron {len(ip_user_status_map)} IPs únicas.")
    # Opcional: Imprimir un pequeño resumen de los datos extraídos
    # for ip, users in list(ip_user_status_map.items())[:3]:
    #     print(f"  IP: {ip}, Data: {users}")

except NameError:
    print("Error: El fichero de log no ha sido cargado. Ejecuta la celda 2 primero.")
except FileNotFoundError:
    print(f"Error: No se encontró el fichero de log '{log_filename}'.")
except Exception as e:
    print(f"Ocurrió un error al leer el fichero de log: {e}")




Análisis del log completado. Se encontraron 13 IPs únicas.


In [ ]:
# @title 5. Análisis con MaxMind (Geolocalización por País)
# @markdown Esta celda utiliza los datos extraídos en el paso anterior y los enriquece con información de geolocalización por país de la base de datos de MaxMind.

import geoip2.database
from prettytable import PrettyTable

geoip_results = [] # Lista para guardar los resultados para la fusión final
if 'ip_user_status_map' in locals() and ip_user_status_map:
    print(f"Geolocalizando {len(ip_user_status_map)} IPs únicas con MaxMind...")
    geoip_table = PrettyTable()
    geoip_table.field_names = ["IP Address", "Usuario(s) y Estado(s)", "País (GeoIP)"]
    geoip_table.align["IP Address"] = "l"
    geoip_table.align["Usuario(s) y Estado(s)"] = "l"

    try:
        geoip_reader = geoip2.database.Reader(db_filename)
        for ip, users_data in sorted(ip_user_status_map.items()):
            # Formateamos la información de usuarios y estados
            user_status_parts = []
            for user, statuses in sorted(users_data.items()):
                status_str = ", ".join(sorted(list(statuses)))
                user_status_parts.append(f"{user} ({status_str})")
            user_status_display = "\n".join(user_status_parts)

            row_data = {'IP Address': ip, 'Usuario(s) y Estado(s)': user_status_display}
            try:
                response = geoip_reader.country(ip)
                row_data['País (GeoIP)'] = response.country.name or "No disponible"
            except geoip2.errors.AddressNotFoundError:
                row_data['País (GeoIP)'] = "No encontrado en DB"

            geoip_table.add_row([row_data['IP Address'], row_data['Usuario(s) y Estado(s)'], row_data['País (GeoIP)']])
            geoip_results.append(row_data)
        geoip_reader.close()
        print("\n--- Tabla 1: Resultados de Geolocalización (MaxMind) ---")
        print(geoip_table)
    except NameError:
        print("Error: El fichero de base de datos MaxMind no ha sido cargado. Ejecuta la celda 3 primero.")
    except FileNotFoundError:
        print(f"Error: No se encontró el fichero de base de datos '{db_filename}'.")
    except Exception as e:
        print(f"Ocurrió un error al procesar los datos: {e}")
else:
    print("No se encontraron datos para analizar. Ejecuta la celda 4 primero.")




Geolocalizando 13 IPs únicas con MaxMind...

--- Tabla 1: Resultados de Geolocalización (MaxMind) ---
+----------------+--------------------------------------+----------------+
| IP Address     | Usuario(s) y Estado(s)               |  País (GeoIP)  |
+----------------+--------------------------------------+----------------+
| 103.77.161.5   | arodriguez (SUCCESS)                 |    Vietnam     |
| 185.188.61.54  | admin (SUCCESS)                      |     Spain      |
| 193.148.17.149 | arodriguez (SUCCESS)                 | United Kingdom |
| 194.35.233.18  | temp_user (SUCCESS)                  | United Kingdom |
| 195.55.80.122  | agarcia (LOGOFF, SUCCESS)            |     Spain      |
| 209.94.57.14   | admin (FAILED, SUCCESS)              | United States  |
| 212.170.35.201 | pjimenez (SUCCESS)                   |     Spain      |
| 217.12.18.4    | jlopez (SUCCESS)                     |     Spain      |
| 52.186.45.11   | admin (FAILED)                       | United States  

In [ ]:
# @title 6. Análisis con RDAP (Propietario de la Red)
# @markdown Esta celda enriquece los datos iniciales con información de propiedad de la red obtenida a través de consultas RDAP.

from ipwhois.ipwhois import IPWhois
from ipwhois.exceptions import IPDefinedError

rdap_results = [] # Lista para guardar los resultados para la fusión final
if 'ip_user_status_map' in locals() and ip_user_status_map:
    print(f"Obteniendo información RDAP para las {len(ip_user_status_map)} IPs únicas...")
    rdap_table = PrettyTable()
    rdap_table.field_names = ["IP Address", "Usuario(s) y Estado(s)", "ASN", "Nombre de Red", "Organización Responsable"]
    rdap_table.align["IP Address"] = "l"
    rdap_table.align["Usuario(s) y Estado(s)"] = "l"
    rdap_table.align["Nombre de Red"] = "l"
    rdap_table.align["Organización Responsable"] = "l"

    for ip, users_data in sorted(ip_user_status_map.items()):
        user_status_parts = []
        for user, statuses in sorted(users_data.items()):
            status_str = ", ".join(sorted(list(statuses)))
            user_status_parts.append(f"{user} ({status_str})")
        user_status_display = "\n".join(user_status_parts)

        row_data = {'IP Address': ip, 'Usuario(s) y Estado(s)': user_status_display}
        try:
            obj = IPWhois(ip)
            results = obj.lookup_rdap(depth=1)
            row_data['ASN'] = results.get('asn', 'N/A')
            row_data['Nombre de Red'] = results.get('network', {}).get('name', 'N/A')
            row_data['Organización Responsable'] = results.get('asn_description', 'N/A')
        except IPDefinedError:
            row_data['ASN'] = "-"
            row_data['Nombre de Red'] = "-"
            row_data['Organización Responsable'] = "IP Privada/Reservada"
        except Exception as e:
            row_data['ASN'] = "-"
            row_data['Nombre de Red'] = "-"
            row_data['Organización Responsable'] = "Error en consulta RDAP"

        rdap_table.add_row([row_data['IP Address'], row_data['Usuario(s) y Estado(s)'], row_data['ASN'], row_data['Nombre de Red'], row_data['Organización Responsable']])
        rdap_results.append(row_data)
    print("\n--- Tabla 2: Resultados de Propiedad de Red (RDAP) ---")
    print(rdap_table)
else:
    print("No se encontraron datos para analizar. Ejecuta la celda 4 primero.")




Obteniendo información RDAP para las 13 IPs únicas...

--- Tabla 2: Resultados de Propiedad de Red (RDAP) ---
+----------------+--------------------------------------+--------+------------------------------+--------------------------------------+
| IP Address     | Usuario(s) y Estado(s)               |  ASN   | Nombre de Red                | Organización Responsable             |
+----------------+--------------------------------------+--------+------------------------------+--------------------------------------+
| 103.77.161.5   | arodriguez (SUCCESS)                 | 45544  | PAVN-VN                      | SUPERDATA-AS-VN SUPERDATA-, VN       |
| 185.188.61.54  | admin (SUCCESS)                      | 203020 | Hostroyale_Barcelona_Network | HOSTROYALE, IN                       |
| 193.148.17.149 | arodriguez (SUCCESS)                 |  9009  | M247-LTD-Manchester          | M247, RO                             |
| 194.35.233.18  | temp_user (SUCCESS)                  | 62240  | P

In [ ]:
# @title 7. Fusión de Resultados (GeoIP + RDAP)
# @markdown Esta celda final toma los resultados de las dos celdas de análisis anteriores y los fusiona en una única tabla consolidada para una vista completa.

if 'geoip_results' in locals() and 'rdap_results' in locals():
    print("Fusionando los resultados de MaxMind y RDAP...")

    # Creamos diccionarios para una búsqueda eficiente
    geoip_dict = {item['IP Address']: item for item in geoip_results}
    rdap_dict = {item['IP Address']: item for item in rdap_results}

    # Creamos la tabla final
    merged_table = PrettyTable()
    merged_table.field_names = ["IP Address", "Usuario(s) y Estado(s)", "País (GeoIP)", "ASN", "Nombre de Red", "Organización Responsable"]
    merged_table.align["IP Address"] = "l"
    merged_table.align["Usuario(s) y Estado(s)"] = "l"
    merged_table.align["Nombre de Red"] = "l"
    merged_table.align["Organización Responsable"] = "l"

    for ip in sorted(ip_user_status_map.keys()):
        geoip_row = geoip_dict.get(ip, {})
        rdap_row = rdap_dict.get(ip, {})

        merged_table.add_row([
            ip,
            geoip_row.get('Usuario(s) y Estado(s)', '-'),
            geoip_row.get('País (GeoIP)', '-'),
            rdap_row.get('ASN', '-'),
            rdap_row.get('Nombre de Red', '-'),
            rdap_row.get('Organización Responsable', '-')
        ])

    print("\n--- Tabla Final: Resultados Consolidados ---")
    print(merged_table)
else:
    print("No se encontraron los resultados para fusionar. Asegúrate de ejecutar las celdas 5 y 6 primero.")

Fusionando los resultados de MaxMind y RDAP...

--- Tabla Final: Resultados Consolidados ---
+----------------+--------------------------------------+----------------+--------+------------------------------+--------------------------------------+
| IP Address     | Usuario(s) y Estado(s)               |  País (GeoIP)  |  ASN   | Nombre de Red                | Organización Responsable             |
+----------------+--------------------------------------+----------------+--------+------------------------------+--------------------------------------+
| 103.77.161.5   | arodriguez (SUCCESS)                 |    Vietnam     | 45544  | PAVN-VN                      | SUPERDATA-AS-VN SUPERDATA-, VN       |
| 185.188.61.54  | admin (SUCCESS)                      |     Spain      | 203020 | Hostroyale_Barcelona_Network | HOSTROYALE, IN                       |
| 193.148.17.149 | arodriguez (SUCCESS)                 | United Kingdom |  9009  | M247-LTD-Manchester          | M247, RO              

In [ ]:
# @title 9. Detección de Viajes Imposibles
# @markdown Esta celda analiza todos los accesos de un mismo usuario para detectar "viajes imposibles". Un viaje se considera imposible si la velocidad requerida para viajar entre dos puntos de acceso consecutivos es superior a la de un avión comercial (900 km/h).

import math
from datetime import datetime

# --- Constantes y Funciones ---
MAX_SPEED_KMH = 900 # Velocidad máxima razonable en km/h

def haversine(lat1, lon1, lat2, lon2):
    """Calcula la distancia en km entre dos puntos geográficos."""
    R = 6371  # Radio de la Tierra en km
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    lat1 = math.radians(lat1)
    lat2 = math.radians(lat2)
    a = math.sin(dLat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dLon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# --- Paso 1: Leer todos los eventos de login del log ---
login_events = []
try:
    with open(log_filename, mode='r', encoding='utf-8') as f:
        next(f) # Saltamos la cabecera
        reader = csv.reader(f)
        for row in reader:
            if len(row) >= 4:
                # Solo nos interesan los accesos exitosos para este análisis
                if row[3] == 'SUCCESS':
                    login_events.append({
                        'timestamp': datetime.strptime(row[0], '%Y-%m-%d %H:%M:%S'),
                        'user': row[1],
                        'ip': row[2]
                    })
except NameError:
    print("Error: El fichero de log no ha sido cargado. Ejecuta la celda 2 primero.")
except Exception as e:
    print(f"Error leyendo el fichero de log: {e}")

# --- Paso 2: Agrupar eventos por usuario ---
user_logins = defaultdict(list)
for event in login_events:
    user_logins[event['user']].append(event)

# --- Paso 3: Analizar viajes imposibles ---
if 'db_filename_city' in locals():
    print("Analizando viajes imposibles...")
    impossible_travel_table = PrettyTable()
    impossible_travel_table.field_names = ["Usuario", "Desde IP", "Hasta IP", "Desde Ubicación", "Hasta Ubicación", "Distancia (km)", "Tiempo", "Velocidad (km/h)"]
    impossible_travel_table.align["Velocidad (km/h)"] = "r"

    ip_coords_cache = {} # Caché para no buscar la misma IP múltiples veces

    try:
        city_reader = geoip2.database.Reader(db_filename_city)

        for user, events in user_logins.items():
            # Ordenamos los eventos por tiempo para cada usuario
            sorted_events = sorted(events, key=lambda x: x['timestamp'])

            for i in range(len(sorted_events) - 1):
                event1 = sorted_events[i]
                event2 = sorted_events[i+1]

                # Obtener coordenadas para ambas IPs (usando caché)
                coords = []
                for event in [event1, event2]:
                    if event['ip'] not in ip_coords_cache:
                        try:
                            response = city_reader.city(event['ip'])
                            ip_coords_cache[event['ip']] = {
                                'lat': response.location.latitude,
                                'lon': response.location.longitude,
                                'loc_str': f"{response.city.name}, {response.country.name}"
                            }
                        except geoip2.errors.AddressNotFoundError:
                            ip_coords_cache[event['ip']] = None
                    coords.append(ip_coords_cache[event['ip']])

                coords1, coords2 = coords
                if not coords1 or not coords2:
                    continue # No podemos calcular si falta alguna ubicación

                # Calcular distancia y tiempo
                distance = haversine(coords1['lat'], coords1['lon'], coords2['lat'], coords2['lon'])
                time_delta_seconds = (event2['timestamp'] - event1['timestamp']).total_seconds()

                if time_delta_seconds > 0:
                    time_delta_hours = time_delta_seconds / 3600
                    speed = distance / time_delta_hours

                    if speed > MAX_SPEED_KMH:
                        time_str = f"{int(time_delta_hours)}h {int((time_delta_hours*60)%60)}m"
                        impossible_travel_table.add_row([
                            user,
                            event1['ip'],
                            event2['ip'],
                            coords1['loc_str'],
                            coords2['loc_str'],
                            f"{distance:.0f}",
                            time_str,
                            f"{speed:.0f}"
                        ])

        city_reader.close()
        if impossible_travel_table.rows:
            print("\n--- ¡ALERTA! Se han detectado los siguientes viajes imposibles ---")
            print(impossible_travel_table)
        else:
            print("\nAnálisis completado. No se han detectado viajes imposibles.")

    except NameError:
        print("Error: El fichero de base de datos MaxMind (City) no ha sido cargado. Ejecuta la celda 4 primero.")
    except FileNotFoundError:
        print(f"Error: No se encontró el fichero de base de datos '{db_filename_city}'.")
    except Exception as e:
        print(f"Ocurrió un error durante el análisis: {e}")
else:
    print("No se puede realizar el análisis de viajes imposibles sin la base de datos de ciudades.")

Analizando viajes imposibles...

--- ¡ALERTA! Se han detectado los siguientes viajes imposibles ---
+------------+---------------+----------------+---------------------+----------------------------+----------------+--------+------------------+
|  Usuario   |    Desde IP   |    Hasta IP    |   Desde Ubicación   |      Hasta Ubicación       | Distancia (km) | Tiempo | Velocidad (km/h) |
+------------+---------------+----------------+---------------------+----------------------------+----------------+--------+------------------+
| arodriguez |  88.2.55.190  |  103.77.161.5  |   Barcelona, Spain  |       None, Vietnam        |     10076      | 0h 9m  |            63195 |
| arodriguez |  103.77.161.5 |  88.2.55.190   |    None, Vietnam    |      Barcelona, Spain      |     10076      | 7h 5m  |             1422 |
| arodriguez | 89.108.83.167 | 193.148.17.149 |     None, Russia    | Manchester, United Kingdom |      2549      | 2h 45m |              926 |
|   admin    |  209.94.57.14 | 185.1